# From DataBase to DataFrame with sqlite3 and pandas

I recently have completed a LinkedIn course on basic SQL with SQLITESTUDIO. I am going to review some of the lesssons from that course here with python package sqlite3. To do that, let us first load the necessary packages.

In [1]:
import pandas as pd
from sqlite3 import connect

This sqlite3 package, along with pandas, are handy tools to run SQL command in python environment. This package requires database (.db) file. Therefore, if the exported file from the database is in other formats, for example, .sql, we can use sqlite3 to transform those in a .db file. I have used following steps to convert .sql file into .db file using sqlite3:

- In command prompt, change the directory (cd) to sqlite3.exe
- Write a database file by command: >sqlite3 databasename.db
- Paste the databasename.db in the same directory by the command: >.database  
- Copy the .sql file in the same directory.
- Read the .sql file (databasename.sql) and overwrite on the .db file by the command: >.read databasename.sql

We can now load the database with the "connect" fucntion from sqlite3.

In [2]:
conn = connect("album.db") #loading album database
pd.read_sql('SELECT name from sqlite_master where type= "table";', conn) # First of all, let us list all the tables

,name
0,album
1,track


In [3]:
pd.read_sql('''SELECT * FROM album;''', conn).head(5) # let's have a quick look at the album table

,id,title,artist,label,released
0,1,Two Men with the Blues,Willie Nelson and Wynton Marsalis,Blue Note,2008-07-08
1,11,Hendrix in the West,Jimi Hendrix,Polydor,1972-01-00
2,12,Rubber Soul,The Beatles,Parlophone,1965-12-03
3,13,Birds of Fire,Mahavishnu Orchestra,Columbia,1973-03-00
4,16,Live And,Johnny Winter,Columbia,1971-05-00


In [4]:
pd.read_sql('''SELECT * FROM track;''', conn).head(5) # and the track table

,id,album_id,title,track_number,duration
0,1,1,Bright Lights Big City,1,320
1,2,1,Night Life,2,344
2,3,1,Basin Street Blues,5,296
3,4,1,Caldonia,3,205
4,5,1,Stardust,4,308


So, the album table shows title of the album, artist names, release date etc. On the other hand, the track table shows title of the tracks, duration etc. Now, this two tables are related by "album_id" which will be useful whiule joining both the tables. Let's say that we are interested to find out how many tracks are in a particular album along with the duration of the albums arranged in descending order.

In [5]:
df = pd.read_sql('''SELECT a.title AS Album, COUNT(t.track_number) as Tracks, 
    ((SUM(t.duration)/60) || ':' || substr('00' || (SUM(t.duration)%60), -2, 2)) AS Duration
  FROM track AS t
  JOIN album AS a
    ON a.id = t.album_id
  GROUP BY a.id
  ORDER BY Duration DESC, Album''', conn)

df

,Album,Tracks,Duration
0,Two Men with the Blues,10,53:27
1,Hendrix in the West,9,49:30
2,Kind of Blue,5,45:54
3,Live And,6,40:32
4,Birds of Fire,10,40:24
5,Rubber Soul,14,35:39
6,Apostrophe,9,31:47


Now let's say that we are interested in findign out information about the tracks that are less than 90 seconds long. To do that we use nested SELECT within JOIN command. It is important to note that, the schema within JOIN lists all the column from correspocning table that we need for our primary query.

In [6]:
df1 = pd.read_sql('''SELECT a.title AS album, a.artist, t.track_number AS seq, t.title, t.duration AS secs
  FROM album AS a
  JOIN (
    SELECT DISTINCT album_id, track_number, duration, title
      FROM track
      WHERE duration <= 90
  ) AS t
    ON t.album_id = a.id
  ORDER BY a.title, t.track_number;''', conn)

df1

,album,artist,seq,title,secs
0,Birds of Fire,Mahavishnu Orchestra,4,Sapphire Bullets of Pure Love,24
1,Hendrix in the West,Jimi Hendrix,6,Sgt. Pepper's Lonely Hearts Club Band,76
